loops, mergind and appending, missing data

### MOINDRES CARRES ORDINAIRES
On commence par charger les mêmes données que dans le TD3, avec les mêmes codes strictement (à un petit détail près, cf. note en bas de code):

In [14]:
#ouverture des données:
library(foreign) #pour ouvrir les données dta de version 14 ou inférieure
anes <- read.dta("TD3_anescum_small_v12.dta")

#on garde seulement quelques variables et on les renomme pour plus de clarté:
library(plyr)
library(dplyr)
anes <- anes %>% 
        dplyr::select(year = VCF0004,
               age = VCF0101,
               gender = VCF0104,
               race = VCF0106,
               edu = VCF0140,
               south = VCF0113,
               income = VCF0114,
               partyid = VCF0301,
               interest = VCF0310,
               govtrust = VCF0604,
               abortion = VCF0838,
               demtherm = VCF0218)
head(anes)

#note: ici, "select" est précédé de "dplyr::". pourquoi ? car nous chargeons aussi le package "arm" qui a aussi sa propre fonction "select" et qui va venir dominer la fonction "select" du package "dplyr". 
#il faut donc préciser de quel package on nécessite la fonction "select"

year,age,gender,race,edu,south,income,partyid,interest,govtrust,abortion,demtherm
1948,NA,1. Male,1,NA,NA,3. 34 to 67 percentile,NA,NA,NA,NA,NA
1948,NA,2. Female,1,NA,NA,5. 96 to 100 percentile,NA,NA,NA,NA,NA
1948,NA,2. Female,1,NA,NA,4. 68 to 95 percentile,NA,NA,NA,NA,NA
1948,NA,2. Female,1,NA,NA,5. 96 to 100 percentile,NA,NA,NA,NA,NA
1948,NA,1. Male,1,NA,NA,4. 68 to 95 percentile,NA,NA,NA,NA,NA
1948,NA,2. Female,1,NA,NA,5. 96 to 100 percentile,NA,NA,NA,NA,NA


En plus de **foreign**, **plyr**, **dplyr**, on va avoir besoin des packages : **ggplot2** et un nouveau package: **arm** qui permet de montrer les résultats MCO plus lisiblement et d'extraire les écart-types :

In [7]:
library(arm)
library(ggplot2)

Loading required package: MASS

Attaching package: ‘MASS’

The following object is masked from ‘package:dplyr’:

    select

Loading required package: Matrix
Loading required package: lme4

arm (Version 1.9-3, built: 2016-11-21)

Working directory is /workspace



# 1. Modèle de régression simple
Nous allons maintenant estimer un modèle simple par MCO et nous allons en donner une représentation graphique (de plus en plus courant). Un modèle simple est un modèle qui met en relation une variable dépendante y avec une explicative x.

### 1.1 Estimation
Nous souhaitons savoir si les préférences pour les démocrates varient selon le genre. 
Commençons par regarder les variables:

In [ ]:
#on recode gender
levels(anes$gender)
levels(anes$gender) <- c(NA, "Homme", "Femme")
#on inspecte dermtherm et gender
summary(anes$demtherm)
summary(anes$gender)

Nous régressons ensuite *gender* sur *demtherm* (qui varie de 0 à 100), grâce à **lm()** (lm(y~x)).

In [ ]:
#on régresse grâce à lm()
lm1 <- lm(demtherm ~ gender, data = anes)
display(lm1)

### 1.2 Représentation graphique
A partir de cette estimation on peut faire deux types de représentations graphiques: 

- l'effet moyen d'être une femme (relativement à l'homme)
- l'opinion moyenne d'un homme et l'opinion moyenne d'une femme (la différence étant l'effet moyen d'être d'un genre plutôt que d'un autre)

L'option 1 revient à représenter le coefficient de *gender* estimé par MCO. 

L'option 2 revient à représenter la valeur prédite moyenne de l'opinion des hommes d'un côté et des femmes de l'autre.

### 1.2.1  Option 1 : représenter le coefficient

On peut extraire les coefficients avec **coef()**, les écart-types avec **se.coef()**, et l'intervalle de confiance avec **confint()**.

In [ ]:
coef(lm1)

In [ ]:
coef(lm1)[2]

In [ ]:
se.coef(lm1)
se.coef(lm1)[2]

In [ ]:
confint(lm1, level=0.95)[1]

In [ ]:
confint(lm1, level=0.95)

On peut donc les utiiser pour créer un data frame avec toutes les informations nécessaires à contruire un graphique de notre résultat principal. Nous avons besoin: 
- du coefficient de 'gender'
- de l'intervalle de confiance autour de ce coefficient
- que nous devons intégrer un à dataframe

In [ ]:
#coefficient de gender
coef(lm1)[2]
#intervalle de confiance: 
intervgender<-coef(lm1)[2]+c(-1,1)*se.coef(lm1)[2]*1.96 #option1
intervgender
intervgender<-confint(lm1,level=0.95)[2,]  #option2
intervgender
#création du data frame:
model1 <- data.frame(coef=coef(lm1)[2],
                    borneinf=intervgender[1],
                    bornesup=intervgender[2],
                    model="Modèle simple")
#le dataframe crée:
model1
model1[1]

On peut ensuite réaliser le graphique avec **ggplot()**.

In [ ]:
ggplot(model1, aes(x=model,y=coef)) +
    geom_point()

On ajoute l'intervalle de confiance :

In [ ]:
ggplot(model1, aes(x=model,y=coef)) +
    geom_point() + 
    geom_errorbar(aes(ymin = borneinf, ymax = bornesup), width = 0.1) 
    

On change l'échelle de y pour améliorer la lecture du graphique:

In [ ]:
ggplot(model1, aes(x=model,y=coef)) +
    geom_point() + 
    geom_errorbar(aes(ymin = borneinf, ymax = bornesup), width = 0.1) +
    geom_hline(yintercept = 0, lty = 2, color = "red") 

On ajoute des labels aux axes :

In [ ]:
ggplot(model1, aes(x=model,y=coef)) +
    geom_point() + 
    geom_errorbar(aes(ymin = borneinf, ymax = bornesup), width = 0.1) +
    geom_hline(yintercept = 0, lty = 2, color = "red") +
    xlab("") +
    ylab("Opinion des femmes sur le parti démocrate (relativement aux hommes)")

### 1.2.2 Option 2 : représenter la moyenne conditionnelle de la variable dépendante y selon les différentes catégories de x

On extrait de l'estimation "lm1" les valeurs prédites du score pro démocrate du thermomètre, ainsi que l'intervalle de confiance autour de ces valeurs prédites. 

In [ ]:
pred1 <- predict(lm1,
                 newdata = data.frame(gender = c("Homme", "Femme")),
                 se.fit = T,
                 interval = "confidence")

Regardons ce que ça donne :

In [ ]:
pred1

Ce qui nous intéresse se trouve dans $fit. On crée un nouveau dataframe *pred1* dans leuel on crée la variable *Genre* pour la légende du graphique :

In [ ]:
pred1 <- data.frame(pred1$fit, Genre = c("Hommes", "Femmes"))
pred1

On peut maintenant représenter les coefficients graphiquement par **ggplot** et **geom_point()** :

In [ ]:
ggplot(pred1, aes(x = Genre, y = fit, color = Genre)) +
    geom_point() 

On ajoute les intervalles de confiance (**geom_errorbar()**) et des labels aux axes (**xlab()** et **ylab()**) : 

In [ ]:

ggplot(pred1, aes(x = Genre, y = fit, color = Genre)) +
    geom_point() +
    geom_errorbar(aes(ymin = lwr, ymax = upr), width = 0.1) +
    xlab("") +
    ylab("Moyenne du score pro-démocrate (entre 0 et 100)")

Discussion: Que conclure ? Quelle est la relation entre le premier graphique et le second graphique ?

# 2. Modèle de régression multiple
On peut également estimer un modèle à plusieurs variables explicatives. On ajoute à notre modèle simple deux variables: *income* et *age*.

### 2.1 Estimation

Avant toute chose on vérifie quelle est la nature des données, et on les inspecte pour vérifier que les variables manquantes sont bien encodées NA (not available).

In [16]:
class(anes$age)
class(anes$income)

[1] "numeric"

[1] "factor"

In [17]:
sort(unique(anes$age))
levels(anes$income)

[1]  0 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40
[26] 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65
[51] 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90
[76] 91 92 93 94 95 96 97 98 99

[1] "1. 0 to 16 percentile"   "2. 17 to 33 percentile" 
[3] "3. 34 to 67 percentile"  "4. 68 to 95 percentile" 
[5] "5. 96 to 100 percentile"

On voit que certaines personnes ont un âge de 0. On voit aussi que la catégorie 1 de *income* correspond en fait à de l'information manquante.

Nous ne souhaitons pas que la régression tienne compte des observations pour lesquelles la valeur est en fait manquante. Pour cela, on effectue les modifications nécessaires concernant les valeurs manquantes : 

In [18]:
#correction de age
anes$age[anes$age==0]<-NA
sort(unique(anes$age))

#correction des quintiles
levels(anes$income)[1]<-NA
levels(anes$income)

[1] 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41
[26] 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66
[51] 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91
[76] 92 93 94 95 96 97 98 99

[1] "1. 0 to 16 percentile"   "2. 17 to 33 percentile" 
[3] "3. 34 to 67 percentile"  "4. 68 to 95 percentile" 
[5] "5. 96 to 100 percentile"

On peut alors régresser :

In [23]:
lm2 <- lm(demtherm~gender+income+age, data=anes)

lm(formula = demtherm ~ gender + income + age, data = anes)
                              coef.est coef.se
(Intercept)                    64.00     0.59 
gender2. Female                 3.89     0.30 
income2. 17 to 33 percentile   -3.99     0.51 
income3. 34 to 67 percentile   -8.50     0.45 
income4. 68 to 95 percentile  -12.48     0.47 
income5. 96 to 100 percentile -17.94     0.74 
age                             0.08     0.01 
---
n = 23686, k = 7
residual sd = 22.92, R-Squared = 0.06



Call:
lm(formula = demtherm ~ gender + income + age, data = anes)

Residuals:
    Min      1Q  Median      3Q     Max 
-75.333 -14.160   0.658  17.241  51.333 

Coefficients:
                                Estimate Std. Error t value Pr(>|t|)    
(Intercept)                    64.004530   0.592598 108.007  < 2e-16 ***
gender2. Female                 3.890932   0.302843  12.848  < 2e-16 ***
income2. 17 to 33 percentile   -3.992904   0.508523  -7.852 4.27e-15 ***
income3. 34 to 67 percentile   -8.501883   0.448869 -18.941  < 2e-16 ***
income4. 68 to 95 percentile  -12.483683   0.470866 -26.512  < 2e-16 ***
income5. 96 to 100 percentile -17.936899   0.735568 -24.385  < 2e-16 ***
age                             0.079978   0.008688   9.206  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 22.92 on 23679 degrees of freedom
  (26074 observations deleted due to missingness)
Multiple R-squared:  0.06226,	Adjusted R-squared:  0.06202 
F-s

Pour visualiser : 

In [24]:
display(lm2) #infos de base

lm(formula = demtherm ~ gender + income + age, data = anes)
                              coef.est coef.se
(Intercept)                    64.00     0.59 
gender2. Female                 3.89     0.30 
income2. 17 to 33 percentile   -3.99     0.51 
income3. 34 to 67 percentile   -8.50     0.45 
income4. 68 to 95 percentile  -12.48     0.47 
income5. 96 to 100 percentile -17.94     0.74 
age                             0.08     0.01 
---
n = 23686, k = 7
residual sd = 22.92, R-Squared = 0.06


In [25]:
summary(lm2) #avec les étoiles


Call:
lm(formula = demtherm ~ gender + income + age, data = anes)

Residuals:
    Min      1Q  Median      3Q     Max 
-75.333 -14.160   0.658  17.241  51.333 

Coefficients:
                                Estimate Std. Error t value Pr(>|t|)    
(Intercept)                    64.004530   0.592598 108.007  < 2e-16 ***
gender2. Female                 3.890932   0.302843  12.848  < 2e-16 ***
income2. 17 to 33 percentile   -3.992904   0.508523  -7.852 4.27e-15 ***
income3. 34 to 67 percentile   -8.501883   0.448869 -18.941  < 2e-16 ***
income4. 68 to 95 percentile  -12.483683   0.470866 -26.512  < 2e-16 ***
income5. 96 to 100 percentile -17.936899   0.735568 -24.385  < 2e-16 ***
age                             0.079978   0.008688   9.206  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 22.92 on 23679 degrees of freedom
  (26074 observations deleted due to missingness)
Multiple R-squared:  0.06226,	Adjusted R-squared:  0.06202 
F-s

Discussion : interpréter les coefficients estimés par le modèle lm2.

### 2.2 Représentation graphique